In [ ]:
 #!pip install vllm
 #!pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/
 #!pip install datasets

In [ ]:
import os
import shutil

# Eliminar caché anterior (¡CUIDADO! esto borra modelos descargados)
cache_dir = "/root/.cache/huggingface"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    print(f"Caché eliminado: {cache_dir}")

# Configurar ANTES de cualquier importación
os.environ['HF_HOME'] = '/workspace'
os.environ['TRANSFORMERS_CACHE'] = '/workspace/transformers'
os.environ['HF_HUB_CACHE'] = '/workspace/hub'

# Crear directorios
os.makedirs('/workspace/transformers', exist_ok=True)
os.makedirs('/workspace/hub', exist_ok=True)

# REINICIA EL KERNEL del notebook después de esto

In [ ]:
"""
from vllm import LLM, SamplingParams

# Inicializar el modelo
llm = LLM(
    model="carlosvillu/gemma2-9b-teacher-eval-nota-feedback",
    gpu_memory_utilization=0.95,  # Usar más memoria disponible
    max_model_len=1024,  # Reducir un poco desde 8192
    tensor_parallel_size=1,
    dtype="bfloat16"  # Usar float16 para eficiencia
)

# Configurar parámetros de generación
sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.95,
    max_tokens=50
)
"""

In [ ]:
import os
import torch
from vllm import LLM, SamplingParams

# Optimizaciones de entorno para L4 (sin FlashInfer)
os.environ["VLLM_USE_PRECOMPILED"] = "1"

# Limpiar caché GPU antes de cargar
torch.cuda.empty_cache()

# Configuración optimizada para NVIDIA L4
llm = LLM(
    model="carlosvillu/gemma2-9b-teacher-eval-nota-feedback",
    gpu_memory_utilization=0.90,  # Aumentado para asegurar KV cache
    max_model_len=1024,           # Reducido para liberar memoria
    tensor_parallel_size=1,
    dtype="bfloat16",
    enable_prefix_caching=True,
    disable_log_stats=True,       # Menos logging para mejor rendimiento
    block_size=16,               # Optimizado para L4
    max_num_seqs=128,            # Más secuencias concurrentes
    compilation_config={
        "level": 2,              # Compilación más rápida que level 3
        "use_inductor": True,
        "use_cudagraph": True,
        "cache_dir": "/tmp/vllm_cache"  # Reutilizar compilaciones
    }
)

# Parámetros de generación optimizados
sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.95,
    max_tokens=200,              # Más tokens de salida
    skip_special_tokens=True
)

print("✅ Modelo cargado y optimizado para NVIDIA L4")
print(f"📊 Memoria GPU utilizada: ~17GB")
print(f"🚀 Listo para inferencia rápida")

In [ ]:
def format_prompts_correctly(examples):
    """
    Formato mejorado: nota única con feedback contextual
    """
    texts = []
    
    # Procesar cada ejemplo en el batch
    for i in range(len(examples["pregunta"])):
        # Obtener evaluaciones
        evaluaciones = [
            examples.get("evaluacion_1", [None])[i],
            examples.get("evaluacion_2", [None])[i], 
            examples.get("evaluacion_3", [None])[i]
        ]
        evaluaciones = [e for e in evaluaciones if e is not None]
        
        if not evaluaciones:
            continue
        
        # Calcular nota final (mediana es más robusta que promedio)
        nota_final = round(statistics.median(evaluaciones))
        curso = examples["curso"][i]
        
        # Generar feedback basado en nota Y nivel educativo
        if nota_final == 0:
            feedback = f"Molt per sota del nivell esperat per a {curso}. Cal reescriure completament amb més contingut i estructura."
        elif nota_final == 1:
            feedback = f"Per sota del nivell de {curso}. Necessita millores significatives en desenvolupament i claredat."
        elif nota_final == 2:
            feedback = f"Just acceptable per a {curso}. Cal ampliar les idees i millorar l'expressió."
        elif nota_final == 3:
            feedback = f"Adequat pel nivell de {curso}. Compleix les expectatives bàsiques del curs."
        elif nota_final == 4:
            feedback = f"Bon treball per a {curso}. Supera les expectatives amb bon desenvolupament."
        else:  # nota_final == 5
            feedback = f"Excel·lent per a {curso}. Molt per sobre del nivell esperat, amb idees complexes ben expressades."
        
        # Instrucciones claras para el modelo
        instruction = """Ets una professora experimentada avaluant textos d'estudiants catalans.
Has d'avaluar amb una nota de 0 a 5 i proporcionar feedback constructiu.

Escala d'avaluació:
0 = Molt per sota del nivell
1 = Per sota del nivell 
2 = Just acceptable
3 = Nivell esperat
4 = Per sobre del nivell
5 = Excel·lent

Respon NOMÉS amb JSON: {"nota": X, "feedback": "..."}"""
        
        # Contenido del usuario
        user_content = f"""{instruction}

Alumne de {curso} respon a "{examples["pregunta"][i]}":
{examples["respuesta"][i]}"""
        
        # Respuesta esperada del asistente
        assistant_response = f'{{"nota": {nota_final}, "feedback": "{feedback}"}}'
        
        # Formato Gemma correcto
        text = f"""<start_of_turn>user
{user_content}<end_of_turn>
<start_of_turn>model"""
        
        texts.append(text)
    
    return {"text": texts}

In [ ]:
# Celda 4: Preparación del dataset - FORMATO CORRECTO PARA GEMMA
from datasets import load_dataset
import statistics


dataset_train = load_dataset("carlosvillu/training-texts", split="train")
dataset_test = load_dataset("carlosvillu/training-texts", split="test")

# Aplicar formato
dataset_train = dataset_train.map(
    format_prompts_correctly,
    batched=True,
    remove_columns=dataset_train.column_names
)

dataset_test = dataset_test.map(
    format_prompts_correctly,
    batched=True,
    remove_columns=dataset_test.column_names
)

print(f"✅ Dataset formateado: {len(dataset_train)} train, {len(dataset_test)} test")
print(f"Ejemplo de formato:\n{dataset_train[0]['text']}")

In [ ]:
# Hacer inferencia
prompts = dataset_test['text']

outputs = llm.generate(prompts, sampling_params)

# Mostrar resultados
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt}")
    print(f"Generated: {generated_text}")
    print("-" * 50)